# Judgments and Feature Logging

In this notebook, we cover the first two steps of Learning to Rank. First we grade some documents as relevant/irrelevant for queries, what we call _judgments_. Second, we retrieve some _features_ - metadata about each graded document in our judgments. We call the process of extracting the features from Solr _feature logging_

NOTE: This notebook depends upon TheMovieDB dataset. If you have any issues, please rerun the [Setting up TheMovieDB notebook](1.ch10-setup-TheMovieDB.ipynb)

In [ ]:
import requests
import sys
sys.path.append('..')
from ltr.client.solr_client import SolrClient

client = SolrClient(host='http://aips-solr:8983/solr')

## Listing 10.3 

A single judgment, grading document 37799 ("The Social Network") as relevant (`grade=1`) for the search query string `social network`

In [ ]:
from ltr.judgments import Judgment

Judgment(grade=1, keywords='social network', doc_id=37799)

## Listing 10.4

A bit bigger judgment list. Here two query strings are graded: `social network` and `star wars`. For `social network` a single movie is graded as relevant, three are irrelevant. Two movies are graded as relevant for `star wars`, three others graded as irrelevant.

In [ ]:
mini_judg_list=[
    # for 'social network' query
    Judgment(grade=1, keywords='social network', doc_id='37799'), #The Social Network
    Judgment(grade=0, keywords='social network', doc_id='267752'), # #chicagoGirl
    Judgment(grade=0, keywords='social network', doc_id='38408'), # Life As We Know It
    Judgment(grade=0, keywords='social network', doc_id='28303'), # The Cheyenne Social Club
    
    # for 'star wars' query
    Judgment(grade=1, keywords='star wars', doc_id='11'), # star wars
    Judgment(grade=1, keywords='star wars', doc_id='1892'), # return of jedi
    Judgment(grade=0, keywords='star wars', doc_id='54138'),# Star Trek Into Darkness
    Judgment(grade=0, keywords='star wars', doc_id='85783'), # The Star
    Judgment(grade=0, keywords='star wars', doc_id='325553'), # Battlestar Galactica
]

mini_judg_list

## Listing 10.5

Demonstrating we have no features for any of our judgments.

In [ ]:
mini_judg_list[0].features


## Listing 10.6

Create a feature set, the first feature retrieves the relevance score of the search string in the `title` field (hence `title:(${keywords})`), the second feature the same for `overview`, finally the third feature is simply the `release_year` of the movie. 

We create a feature store named `movies` in Solr. We'll use the feature store name as a handle when we want to log features farther down.

In [ ]:
requests.delete('http://aips-solr:8983/solr/tmdb/schema/feature-store/movies')

import requests

feature_set = [
    {
      "name" : "title_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    {
      "name" : "release_year",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}release_year"
}}]

resp = requests.put('http://aips-solr:8983/solr/tmdb/schema/feature-store',
                    json=feature_set)
resp.text

# Listing 10.7 & 10.8

The code here is Listing 10.7, the output is Listing 10.8

Recall we have one relevant and three irrelevant movies for `social network`. Here we retrieve all three features created above for each of the four movies. The special `[features..`], tells Solr to append the features from `movies` feature store using the template param `efi.keywords="social network"` in each document.

In [ ]:
import requests

logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"social network\"]",
    'q': "id:37799 OR id:267752 OR id:38408 OR id:28303", #social network graded documents
    'rows': 10,
    'wt': 'json'  
}

solr_resp = requests.post('http://aips-solr:8983/solr/tmdb/select',
                     data=logging_solr_query)

solr_resp.json()

## Omitted from book - parse features Solr response

**The following code is used to generate Listings below. But this parsing code is omitted from the book itself.**

This code simply looks at the solr_resp and populates the corresponding judgments feature vector with the `[features]` from the corresponding solr document.

In [ ]:
def populate_features_for_qid(qid, solr_resp, judg_list):
    solr_json = solr_resp.json()
    doc_id_to_features = {}

    # Map Doc Id => Features
    for doc in solr_json['response']['docs']:
        # Parse '[features] array', ie
        # title_bm25=0.0,overview_bm25=13.237938,vote_average=7.0'
        features = doc['[features]']
        features = features.split(',')
        features = [float(ftr.split('=')[1]) for ftr in features]

        doc_id_to_features[doc['id']] = features

    # Save in correct judgment
    for judgment in judg_list:
        if judgment.qid == qid:
            try:
                judgment.features = doc_id_to_features[judgment.doc_id]
            except KeyError:
                pass




## Listing 10.9 (output)

Listing 10.9 is the output of the following, the resulting processing of logging just for `social network`

In [ ]:
populate_features_for_qid(qid=1, solr_resp=solr_resp, judg_list=mini_judg_list)
mini_judg_list

## Listing 10.10 (output)

Listing 10.10 is the output of the following, which adds features parsed from the `star wars` movies to our judgment list.

In [ ]:
logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"star wars\"]",
    'q': "id:11 OR id:1892 OR id:54138 OR id:85783 OR id:325553", #star wars graded documents
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://aips-solr:8983/solr/tmdb/select',
                     data=logging_solr_query)

populate_features_for_qid(qid=2, solr_resp=resp, judg_list=mini_judg_list)

mini_judg_list

## Loading / logging training set (omitted from book)

The following downloads a larger judgment list, parses it, and logs features for each graded document. It just repeats the full logging workflow in this notebook but in one loop.

In [ ]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby
from ltr import download

judgments='http://es-learn-to-rank.labs.o19s.com/ai_pow_search_judgments.txt'
download([judgments], dest='data/')

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')

with judgments_open('data/ai_pow_search_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))
    
ftr_logger.logged

## Next up, prep data for training

We now have a dataset extracted from the search engine. Next we need to perform some manipulation to the training data. This manipulation turns our slightly strange looking ranking problem into one that looks more like any-other boring machine learning problem.

So when you're ready [please jump into the next section](3.ch10-pairwise-transform.ipynb)